# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation. To run this example Julia's `PyCall` package
needs to be able to find an installation of `pymatgen`.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

┌ Warning: `vendor()` is deprecated, use `BLAS.get_config()` and inspect the output instead
│   caller = npyinitialize() at numpy.jl:67
└ @ PyCall /home/runner/.julia/packages/PyCall/L0fLP/src/numpy.jl:67


Setup an LDA model and discretize using
a single k-point and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=(1, 1, 1))

PlaneWaveBasis discretization:
    Ecut                 : 5.0 Ha
    fft_size             : (32, 32, 32)
    kgrid type           : Monkhorst-Pack
    kgrid                : [1, 1, 1]
    num. irred. kpoints  : 1

    Discretized Model(lda_xc_teter93, 3D):
        lattice (in Bohr)    : [0         , 10.2631   , 10.2631   ]
                               [10.2631   , 0         , 10.2631   ]
                               [10.2631   , 10.2631   , 0         ]
        unit cell volume     : 2162.1 Bohr³
    
        atoms                : Si₁₆
        atom potentials      : ElementPsp(Si, psp=hgh/lda/si-q4)
    
        num. electrons       : 64
        spin polarization    : none
        temperature          : 0 Ha
    
        terms                : Kinetic()
                               AtomicLocal()
                               AtomicNonlocal()
                               Ewald()
                               PspCorrection()
                               Hartree()
            

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.125727e+02     1.563218e+00
 * time: 0.6073718070983887
     1     1.086974e+01     8.998273e-01
 * time: 1.5577099323272705
     2    -1.127478e+01     1.044311e+00
 * time: 2.0381698608398438
     3    -3.313016e+01     7.573885e-01
 * time: 2.6840219497680664
     4    -4.622845e+01     5.948360e-01
 * time: 3.3372409343719482
     5    -5.649370e+01     2.083150e-01
 * time: 4.0079028606414795
     6    -5.957153e+01     1.687396e-01
 * time: 4.438501834869385
     7    -6.088119e+01     6.460664e-02
 * time: 4.870097875595093
     8    -6.138391e+01     4.052264e-02
 * time: 5.302686929702759
     9    -6.166642e+01     2.864179e-02
 * time: 5.7548089027404785
    10    -6.184538e+01     2.682137e-02
 * time: 6.185622930526733
    11    -6.195897e+01     3.441458e-02
 * time: 6.61732292175293
    12    -6.204375e+01     2.330895e-02
 * time: 7.049807786941528
    13    -6.211522e+01     1.399016e-02
 * time: 7.481406927108765
 

In [5]:
scfres.energies

Energy breakdown (in Ha):
    Kinetic             25.7671061
    AtomicLocal         -18.8557637
    AtomicNonlocal      14.8522619
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485358 
    Xc                  -19.3336815

    total               -62.261666460968